In [68]:
from typing import Any

import numpy as np
import bloqade.types
from kirin.ir import Method
from bloqade.types import Qubit, MeasurementResult

# Some types we will use, useful for type hints
from kirin.dialects.ilist import IList

from bloqade import squin

Register = IList[Qubit, Any]

from bloqade.pyqrack import StackMemorySimulator, DynamicMemorySimulator

In [69]:
def ghz_constant_depth(n_qubits: int):

    @squin.kernel
    def main() -> Register:
        qreg = squin.qalloc(n_qubits)
        ancilla = squin.qalloc(n_qubits - 1)

        for i in range(n_qubits):
            squin.h(qreg[i])

        for i in range(n_qubits - 1):
            squin.cx(qreg[i], ancilla[i])
        for i in range(n_qubits - 1):
            squin.cx(qreg[i + 1], ancilla[i])

        parity: int = 0
        bits = squin.broadcast.measure(ancilla)
        for i in range(n_qubits - 1):
            parity = parity ^ bits[i]
            if parity == 1:
                squin.x(qreg[i + 1])
        return qreg

    return main


# %%
# At this point, you know the drill. We can simulate this with multirun via PyQrack
emulator = StackMemorySimulator(min_qubits=7)
task = emulator.task(ghz_constant_depth(3))

state = task.batch_state(shots=1000, qubit_map=lambda x: x)
# Even though there is measurement and feedforward, the final state is still pure. Neat!
print(state.eigenvalues)
# %% [markdown]

[1.00000007]


In [70]:
def MSD_encoding(theta, phi,basis="z"):

    @squin.kernel
    def parameterized_MSD_encoding():
        q = squin.qalloc(7)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
    
    return parameterized_MSD_encoding

In [75]:
def MSD_encoding_plus_ancilla(theta, phi,lam, phys_qubits = 7, basis="z"):
    @squin.kernel
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        squin.u3(theta,phi,lam,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
        
    @squin.kernel
    def circ():
        q=squin.qalloc(14)
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam) # Injected state encoding
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi) # Z-basis ancilla encoding

        squin.cx(q[0],q[7])
        squin.cx(q[1],q[8])
        squin.cx(q[2],q[9])
        squin.cx(q[3],q[10])
        squin.cx(q[4],q[11])
        squin.cx(q[5],q[12])
        squin.cx(q[6],q[13])

        z_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])

        if z_measurement[0]:
            squin.x([q])

        squin.broadcast.reset([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=0,phi=0,lam=0) # X-basis ancilla encoding

        squin.cx(q[7],q[0])
        squin.cx(q[8],q[1])
        squin.cx(q[9],q[2])
        squin.cx(q[10],q[3])
        squin.cx(q[11],q[4])
        squin.cx(q[12],q[5])
        squin.cx(q[13],q[6])

        squin.broadcast.h([q[7], q[8], q[9], q[10], q[11], q[12], q[13]])

        x_measurement = squin.broadcast.measure([q[7],q[8],q[9],q[10],q[11],q[12],q[13]])

        return q

        # x_measurement: IList[Qubit, N]

        # for i in range(7):
        #     if squin.broadcast.is_one(x_measurement[i]):
        #         l_x[i]=1
        #     else:
        #         l_x[i]=-1
        
        # control_bools = squin.broadcast.is_one(x_measurement)

        # for i in range(7):
        #     if x_measurement[i]:
        #         l_x.append(-1)
        #     else:
        #         l_x.append(1)

        # stabilizers = [[0,2,4,6],[3,4,5,6],[1,2,5,6]]

        # def get_int(bit_list):
        #     y = 0 
        #     for i in range(phys_qubits):
        #         if bit_list[i] == 1:
        #             y += (2**i)
        #     return y
        
        # def get_syndromes(eigenvalues, stabilizers):
        #     syndrome = []
        #     for stab in stabilizers:
        #         eigval = 1
        #         for idx in stab:
        #             eigval *= eigenvalues[idx]
        #         syndrome.append(eigval)
        #         syndrome = tuple(syndrome)
        #         syndrome = get_int(syndrome)
        #     return syndrome
        
        # def update_pauli_corrections_from_syndrome(current_pauli_list, syndrome, syndrome_table):
        #     if syndrome==7:
        #         return
        #     index = syndrome_table[syndrome]
        #     if current_pauli_list[index] == 0:
        #         current_pauli_list[index] = 1
        #     else:
        #         current_pauli_list[index] = 0

        # pauli_x = [0]*7
        # syndrome_table = {3:0, 6:1, 2:2, 5:3, 1:4, 4:5, 0:6}

        # update_pauli_corrections_from_syndrome(pauli_x, get_syndromes(l_x, stabilizers), syndrome_table)

        # for i in range(7):
        #     if pauli_x[i] == 1:
        #         squin.x(q[i])
        
        # # x_measurement = []
        # # x_measurement.append(squin.measure(q[7]))
        # # x_measurement.append(squin.measure(q[8]))
        # # x_measurement.append(squin.measure(q[9]))
        # # x_measurement.append(squin.measure(q[10]))
        # # x_measurement.append(squin.measure(q[11]))
        # # x_measurement.append(squin.measure(q[12]))
        # # x_measurement.append(squin.measure(q[13]))

        # return x_measurement, z_measurement

    return circ

In [76]:
# %%
# At this point, you know the drill. We can simulate this with multirun via PyQrack
emulator = StackMemorySimulator(min_qubits=21)
task = emulator.task(MSD_encoding_plus_ancilla(1,0,0))

state = task.batch_state(shots=1, qubit_map=lambda x: x)
# Even though there is measurement and feedforward, the final state is still pure. Neat!
print(state.eigenvalues)
# %% [markdown]

AttributeError: 'IList' object has no attribute 'is_active'